# Sympy Module for SU(N) Color algebra

### Overview:

This module provides symbolic tools for computing color factors in QCD (SU(N) group theory). It defines SU(N) generator symbols and invariant tensors, along with rules for simplifying expressions involving them. The implementation closely follows the Mathematica package's functionality, including index contraction and decomposition into structure constants and symmetric tensors.

### Symbolic definitions

* Generator T[a, i, j]: Represents the $(T^a)_i{}^j$ generator of SU(N) in the fundamental representation. Here a is an adjoint index (color index in ${1,\dots,N^2-1}$) and i, j are fundamental indices (running from $1$ to $N$). Generators are treated as non-commutative symbols.


* Structure constant f[a, b, c]: The completely antisymmetric structure constants defined by the commutator: $[T^a, T^b] = f^{a b c}, T^c$. (f[a,b,c] is antisymmetric in all indices.)
Symmetric tensor d[a, b, c]: The completely symmetric tensor defined by the anti-commutator: ${T^a, T^b} = \frac{1}{N}\delta^{a b},I + d^{a b c},T^c$. (d[a,b,c] is symmetric in all its indices.)


* Kronecker delta (fundamental) KDf(i, j): $\delta_{ij}$ for fundamental indices. It is commutative and simplifies such that $\mathrm{KDf}(i,i) = N$ (summing a fundamental index yields $N$).


* Kronecker delta (adjoint) KDad(a, b): $\delta^{a b}$ for adjoint indices. It simplifies so that $\mathrm{KDad}(a,a) = N^2 - 1$ (summing an adjoint index yields $N^2-1$).
Trace TrT(...): A function to compute the trace of a product of generators. For example, TrT(a, b, c) corresponds to $\mathrm{Tr}(T^a T^b T^c)$. The trace is evaluated with the above normalization (generators are traceless and $\mathrm{Tr}(T^a T^b) = \delta^{ab}$).


All these symbols are integrated into SymPy's Expr system, with custom LaTeX rendering for clarity. The symbol N is a global SymPy symbol representing the number of colors (treated as an integer).

### Simplification

The module implements the following substitution and simplification rules, as functions in the Mathematica package 

* Generator completeness(Construction of two generators): $T[a,i,j]T[a,k,l] = \delta_{i,l} \delta_{j,k} - \frac{1}{N} - \delta_{i,j} \delta_{k,l}$.Repeated adjoint indices are summed over. This identity is used to contract a pair of identical adjoint indices, producing products of Kronecker Deltas

Example: `T(a,i,j) # T(a,k,l)` simplifies to `KDF(i,l)*KDF*j,k) -(1/n)* KDF(i,j)*KDF(k,l)`

* Structure constants via commutators: If an expression contains the difference of two terms with generators swapped, it is converted using $T[a, i, j] T[b, j, k]-$ $T[b, i, j] T[a, j, k]=f[a, b, c] T[c, i, k]$. 
In other words, $[T^a, T^b] = f^{abc}, T^c$

The code will recognize a commutator pattern and replace it with `f(a,b,c)*T(c,i,k)` accordingly (introducing a summed index `c` ).
Example: `T(a, i, j) * T(b, j, k)-T(b, i, j) * T(a, j, k)` simplifies to `f(a, b, c) * T(c, i$, k)`.

* Symmetric tensors via anti-commutators: Similarly, a sum of terms with generators swapped is handled as $T[a, i, j] T[b, j, k]+T[b, i, j] T[a, j, k]=$ $\frac{1}{N} \delta^{a b} \delta_{i, k}+d[a, b, c] T[c, i, k]$. 

This corresponds to ${T^a, T^b}$ decomposition into the singlet (proportional to $\delta^{ab}$) and the symmetric tensor $d^{ab c}$.

Example: `T(a,i,j)*T(b,j,k) + T(b,i,j)*T(a,j,k)` simplifies to `(1/N)*KDad(a,b)*KDf(i,k) + d(a,b,c)*T(c,i,k)`.

* Delta contraction (fundamental indices): Kronecker deltas contract fundamental indices on generators or on other deltas:

$\delta_{i j},T[a,j,k] = T[a,i,k]$, and $;T[a,i,j],\delta_{j k} = T[a,i,k]$.

$\delta_{i j},\delta_{j k} = \delta_{i k}$.

$\delta_{i j},\delta_{j k} = \delta_{i k}$.

In the code, any occurrence of `KDf(i, j)` will replace an index j in another factor with `i` (effectively performing the sum over j). For example, `KDf(i, j)*T(a, j, k)` simplifies to `T(a, i, k)`. Chained delta contractions will reduce, e.g. `KDf(i,j)*KDf(j,k)` simplifies to `KDf(i,k)`. If a fundamental delta has both indices the same (e.g. `KDf(i,i)`), it produces N.

* Dleta Construction (adjoint indices): Likewose for adjoint indices

$\delta^{a b},T[b,i,j] = T[a,i,j]$, $;T[a,i,j],\delta^{a b} = T[b,i,j]$.

$\delta^{a b},\delta^{b c} = \delta^{a c}$. And $\delta^{a a} = N^2-1$

In code, `KDad(a,b)` will be identical adjoint indices. For example, `KDad(a,b) * f(b,c,d)` simplifies to f(a,c,d). Summing an adjoint index with itself, `KDad(a,a,)`, yields $N^2-1$.

* Contraction of structure constants: The module encodes common SU(N) identities for products of $f$ and $d$

$f^{a m n}f^{b m n} = N,\delta^{ab}$. In expressions, `f(a,b,x)*f(d,b,x)` (summing over two indices b and x) simplifies to `N*KDad(a,d)`.

$d^{a m n}d^{b m n} = \frac{N^2-4}{N},\delta^{ab}$. Accordingly, `d(a,b,x)*d(c,b,x)` simplifies to `((N**2 - 4)/N) * KDad(a,c)`.

Mixed $f$-$d$ contractions vanish: $f^{a m n}d^{b m n} = 0$. So any product like `f(a,b,x)*d(c,b,x)` simplifies to 0.

* Trace of generators: The `TrT` function automatically contracts fundamental indices in a cyclic product and applies the above rules. Important special cases:

$\mathrm{Tr}(T^a) = 0$ (single generator is traceless). So TrT(a) returns 0.

$\mathrm{Tr}(T^a T^b) = \delta^{ab}$. So `TrT(a,b)` yields `KDad(a,b)`, which evaluates to $\delta^{ab}$. In particular, `TrT(a,a)` gives $N^2-1$.

Higher traces will be reduced using the algebra. For example, $\mathrm{Tr}(T^a T^b T^c)$ is proportional to the symmetric tensor $d^{abc}$ (with our normalization, `TrT(a,b,c)` simplifies to `d(a,b,c)`). As another example, $\mathrm{Tr}(T^a T^a T^b T^c) = \Big(N - \frac{1}{N}\Big)\delta^{b c}$, which the rules will produce by successive contractions.

All these simplifications are applied automatically via pattern matching when using the simplify_color function or when calling TrT. Repeated index summation (Einstein summation) is effectively handled by these rules. For instance, any time the same index symbol appears twice (one up, one down position) in a product, the appropriate delta contraction rule will apply.

### Latex output

Each defined symbol has a custom LaTeX representation for clarity:

* `T(a,i,j)` is rendered as $T^{a}_{;i,j}$.

* `f(a,b,c)` is rendered as $f^{a b c}$ (totally antisymmetric).

* `d(a,b,c)` is rendered as $d^{a b c}$ (totally symmetric).

* `KDf(i,j)` is rendered as $\delta_{i j}$.

* `KDad(a,b)` is rendered as $\delta^{a b}$.


Expressions composed of these will display using these notations. For example, the output of `TrT(a,b)` will be $\delta^{a b}$, and `TrT(a,a)` will display as $N^2 - 1$.

### Python implementation: Code

Below is the python implementation using **SymPi**.

In [ ]:
import sympy as sp

# Global symbol for number of colors
N = sp.Symbol('N', positive=True, integer=True)

class TSymbol(sp.Expr):
    """SU(N) fundamental generator T[a, i, j]."""
    is_commutative = False  # generators do not commute
    def __new__(cls, a, i, j):
        a = sp.sympify(a); i = sp.sympify(i); j = sp.sympify(j)
        return sp.Expr.__new__(cls, a, i, j)
    @property
    def a(self):  # adjoint index
        return self.args[0]
    @property
    def i(self):  # fundamental index (row)
        return self.args[1]
    @property
    def j(self):  # fundamental index (column)
        return self.args[2]
    def _latex(self, printer=None):
        # LaTeX as T^{a}_{ij}
        a_str = printer.doprint(self.a) if printer else str(self.a)
        i_str = printer.doprint(self.i) if printer else str(self.i)
        j_str = printer.doprint(self.j) if printer else str(self.j)
        return r"T^{%s}_{%s\,%s}" % (a_str, i_str, j_str)

def T(a, i, j):
    """Factory function for T[a,i,j]."""
    return TSymbol(a, i, j)

class StructureConstant(sp.Expr):
    """Structure constant f[a, b, c], antisymmetric in a,b,c."""
    is_commutative = True
    def __new__(cls, a, b, c):
        a = sp.sympify(a); b = sp.sympify(b); c = sp.sympify(c)
        # Antisymmetry: if any two indices equal, f = 0
        if a == b or b == c or a == c:
            return sp.Integer(0)
        # Enforce canonical order (sorted by symbol name) with sign for permutations
        indices = [a, b, c]
        sorted_idx = sorted(indices, key=lambda x: str(x))
        # Count parity of permutation from original to sorted
        sign = 1
        temp = indices.copy()
        for pos in range(3):
            swap_idx = temp.index(sorted_idx[pos], pos)
            if swap_idx != pos:
                temp[pos], temp[swap_idx] = temp[swap_idx], temp[pos]
                sign *= -1
        obj = sp.Expr.__new__(cls, *sorted_idx)
        return -obj if sign == -1 else obj
    @property
    def a(self): return self.args[0]
    @property
    def b(self): return self.args[1]
    @property
    def c(self): return self.args[2]
    def _latex(self, printer=None):
        a_str = printer.doprint(self.a) if printer else str(self.a)
        b_str = printer.doprint(self.b) if printer else str(self.b)
        c_str = printer.doprint(self.c) if printer else str(self.c)
        return r"f^{%s\,%s\,%s}" % (a_str, b_str, c_str)

def f(a, b, c):
    """Structure constant f[a,b,c]."""
    return StructureConstant(a, b, c)

class SymmetricTensor(sp.Expr):
    """Symmetric tensor d[a, b, c], symmetric in all indices."""
    is_commutative = True
    def __new__(cls, a, b, c):
        a = sp.sympify(a); b = sp.sympify(b); c = sp.sympify(c)
        # No automatic zero for repeated indices (d with repeated indices can be non-zero).
        # Enforce canonical sorted order for consistency.
        sorted_idx = sorted([a, b, c], key=lambda x: str(x))
        return sp.Expr.__new__(cls, *sorted_idx)
    @property
    def a(self): return self.args[0]
    @property
    def b(self): return self.args[1]
    @property
    def c(self): return self.args[2]
    def _latex(self, printer=None):
        a_str = printer.doprint(self.a) if printer else str(self.a)
        b_str = printer.doprint(self.b) if printer else str(self.b)
        c_str = printer.doprint(self.c) if printer else str(self.c)
        return r"d^{%s\,%s\,%s}" % (a_str, b_str, c_str)

def d(a, b, c):
    """Symmetric tensor d[a,b,c]."""
    return SymmetricTensor(a, b, c)

class DeltaFund(sp.Expr):
    """Kronecker delta for fundamental indices, delta[i,j]."""
    is_commutative = True
    def __new__(cls, i, j):
        i = sp.sympify(i); j = sp.sympify(j)
        if i == j:
            # δ[i,i] = N
            return N
        # Canonical order (swap if needed for consistent representation)
        if str(i) > str(j):
            i, j = j, i
        return sp.Expr.__new__(cls, i, j)
    @property
    def i(self): return self.args[0]
    @property
    def j(self): return self.args[1]
    def _latex(self, printer=None):
        i_str = printer.doprint(self.i) if printer else str(self.i)
        j_str = printer.doprint(self.j) if printer else str(self.j)
        return r"\delta_{%s%s}" % (i_str, j_str)

class DeltaAdj(sp.Expr):
    """Kronecker delta for adjoint indices, delta^{a,b}."""
    is_commutative = True
    def __new__(cls, a, b):
        a = sp.sympify(a); b = sp.sympify(b)
        if a == b:
            # δ^{a,a} = N^2 - 1
            return N**2 - 1
        if str(a) > str(b):
            a, b = b, a
        return sp.Expr.__new__(cls, a, b)
    @property
    def a(self): return self.args[0]
    @property
    def b(self): return self.args[1]
    def _latex(self, printer=None):
        a_str = printer.doprint(self.a) if printer else str(self.a)
        b_str = printer.doprint(self.b) if printer else str(self.b)
        return r"\delta^{%s%s}" % (a_str, b_str)

def KDf(i, j):
    """Fundamental Kronecker delta δ[i,j]."""
    return DeltaFund(i, j)

def KDad(a, b):
    """Adjoint Kronecker delta δ^{a,b}."""
    return DeltaAdj(a, b)

# Wild symbols for pattern matching in simplification
A, B, C, D = [sp.Wild(name) for name in ('A','B','C','D')]
I, J, K, L = [sp.Wild(name) for name in ('I','J','K','L')]
X = sp.Wild('X')  # generic wild if needed for sums

# Define core patterns:
patterns = []
# Generator completeness: T(A,I,J)*T(A,K,L) -> δ_{I,L} δ_{J,K} - (1/N) δ_{I,J} δ_{K,L}
patterns.append(( T(A,I,J) * T(A,K,L),
                  KDf(I,L)*KDf(J,K) - (sp.Integer(1)/N)*KDf(I,J)*KDf(K,L) ))
# Delta contractions with T (fund indices):
patterns += [
    ( KDf(I,J) * T(A, J, K),  T(A, I, K) ),
    ( KDf(I,J) * T(A, K, J),  T(A, K, I) ),
    ( T(A, J, K) * KDf(I, J),  T(A, I, K) ),
    ( T(A, K, J) * KDf(I, J),  T(A, K, I) )
]
# Delta contractions among deltas:
patterns += [
    ( KDf(I,J) * KDf(J,K),  KDf(I, K) ),
    ( KDf(J,K) * KDf(I,J),  KDf(I, K) ),
    ( KDad(A,B) * KDad(B,C),  KDad(A, C) ),
    ( KDad(B,C) * KDad(A,B),  KDad(A, C) )
]
# Delta with T (adjoint index):
patterns += [
    ( KDad(A,B) * T(B, I, J),  T(A, I, J) ),
    ( T(A, I, J) * KDad(A,B),  T(B, I, J) )
]
# Delta with structure constants:
patterns += [
    ( KDad(A,B) * f(B, C, D),  f(A, C, D) ),
    ( KDad(A,B) * f(C, B, D),  f(C, A, D) ),
    ( KDad(A,B) * f(C, D, B),  f(C, D, A) ),
    ( f(B, C, D) * KDad(A,B),  f(A, C, D) ),
    ( f(C, B, D) * KDad(A,B),  f(C, A, D) ),
    ( f(C, D, B) * KDad(A,B),  f(C, D, A) )
]
# Delta with symmetric tensor:
patterns += [
    ( KDad(A,B) * d(B, C, D),  d(A, C, D) ),
    ( KDad(A,B) * d(C, B, D),  d(C, A, D) ),
    ( KDad(A,B) * d(C, D, B),  d(C, D, A) ),
    ( d(B, C, D) * KDad(A,B),  d(A, C, D) ),
    ( d(C, B, D) * KDad(A,B),  d(C, A, D) ),
    ( d(C, D, B) * KDad(A,B),  d(C, D, A) )
]
# f-f and d-d contractions:
patterns += [
    ( f(A,B,C) * f(D,B,C),  N * KDad(A, D) ), 
    ( d(A,B,C) * d(D,B,C),  ((N**2 - 4)/N) * KDad(A, D) )
]
# f-d mixed contraction -> 0:
patterns += [
    ( f(A,B,C) * d(D,B,C),  sp.Integer(0) ),
    ( d(A,B,C) * f(D,B,C),  sp.Integer(0) )
]
# Simplify traces of 2 and 3 generators explicitly:
patterns += [
    ( T(A, I, J) * T(B, J, I),  KDad(A, B) ),       # Tr(T^A T^B) = δ^{AB}
    ( T(A, I, J) * T(B, J, K) * T(C, K, I),  d(A, B, C) )  # Tr(T^A T^B T^C) = d^{ABC}
]

def simplify_color(expr):
    """Simplify a color algebra expression using SU(N) rules."""
    expr = sp.sympify(expr)
    # Iteratively apply patterns until no change
    while True:
        new_expr = expr
        for pat, repl in patterns:
            new_expr = new_expr.replace(pat, repl)
        # Handle commutator and anti-commutator patterns as special cases
        comm_match = new_expr.match( T(A,I,J)*T(B,J,K) - T(B,I,J)*T(A,J,K) )
        if comm_match:
            # [T^A, T^B] = f^{ABX} T^X
            dummy = sp.Dummy('c')  # new dummy index for the result
            new_expr = f(comm_match[A], comm_match[B], dummy) * T(dummy, comm_match[I], comm_match[K])
        anticomm_match = new_expr.match( T(A,I,J)*T(B,J,K) + T(B,I,J)*T(A,J,K) )
        if anticomm_match:
            # {T^A, T^B} = (1/N) δ^{AB} I + d^{ABX} T^X
            dummy = sp.Dummy('c')
            new_expr = (sp.Integer(1)/N)*KDad(anticomm_match[A], anticomm_match[B])*KDf(anticomm_match[I], anticomm_match[K]) \
                       + d(anticomm_match[A], anticomm_match[B], dummy)*T(dummy, anticomm_match[I], anticomm_match[K])
        if new_expr == expr:
            break
        expr = new_expr
    # Expand products over sums for a final simplified form
    return expr.expand()

def TrT(*adj_indices):
    """Trace of a product of generators: Tr(T^{a} T^{b} ...)."""
    if len(adj_indices) == 0:
        return sp.Integer(0)
    if len(adj_indices) == 1:
        # Tr(T^a) = 0
        return sp.Integer(0)
    # Construct the matrix product with dummy fundamental indices
    i0 = sp.Dummy('i')
    idx_prev = i0
    expr = 1
    for A_idx in adj_indices[:-1]:
        idx_next = sp.Dummy('i')
        expr *= T(A_idx, idx_prev, idx_next)
        idx_prev = idx_next
    # Last factor closes the trace
    lastA = adj_indices[-1]
    expr *= T(lastA, idx_prev, i0)
    # Simplify the resulting expression
    return simplify_color(expr)


### Usage Example

Below are a few examples

In [ ]:
from color_algebra import T, f, d, KDf, KDad, TrT, simplify_color, N
import sympy as sp

# 定义一些索引，使用 Sympy 符号
a, b, c, i, j, k = sp.symbols('a b c i j k')

# 示例 1：相同伴随指标的收缩
expr = T(a, i, j) * T(a, j, k)
print(simplify_color(expr))
# 输出: $\delta_{i k} - \frac{1}{N}\,\delta_{i j}\,\delta_{j k}$

# 示例 2：交换子转化为结构常数
expr = T(a, i, j) * T(b, j, k) - T(b, i, j) * T(a, j, k)
print(simplify_color(expr))
# 输出: $f^{a b c}\,T^{c}_{i k}$

# 示例 3：反交换子转化为对称张量
expr = T(a, i, j) * T(b, j, k) + T(b, i, j) * T(a, j, k)
print(simplify_color(expr))
# 输出: $\frac{1}{N}\,\delta^{a b}\,\delta_{i k} + d^{a b c}\,T^{c}_{i k}$

# 示例 4：两个生成元的迹
print(TrT(a, b))
# 输出: $\delta^{a b}$

# 示例 5：三个生成元的迹
print(TrT(a, b, c))
# 输出: $d^{a b c}$

# 示例 6：f 和 d 张量的收缩
print(simplify_color(f(a, b, c) * f(d, b, c)))
# 输出: $N\,\delta^{a d}$

print(simplify_color(f(a, b, c) * d(d, b, c)))
# 输出: $0$